In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import pandas as pd
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
from stop_word_list import *
from tqdm import tqdm
import sys  
import contractions

# Dataset

In [3]:
data = pd.read_csv('./processing_datasets/preprocessed_word_correct.csv', encoding='utf-8')
data.head()

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'..."
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '..."
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ..."


In [4]:
ngrams_reviews = []
for content in tqdm(data['correctmapping']):
    content=content.replace("]","")
    content=content.replace("[","")
    content=content.replace(",","")
    content=content.replace("'","")
    ngrams_reviews.append(content.strip())

100%|██████████████████████████████████████████████████████████████████████| 163847/163847 [00:00<00:00, 418035.31it/s]


In [5]:
data['ngrams_reviews']=ngrams_reviews
data.head()

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...


# bigram & trigram

In [6]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([comment.split() for comment in data.ngrams_reviews])
# Filter only those that occur at least 2 times
finder.apply_freq_filter(200)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

bigram_pmi = pd.DataFrame(bigram_scores,columns= ['bigram', 'pmi'])
#bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

In [7]:
def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in nouns_list or bigram[1] in nouns_list:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True

In [8]:
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 1, axis = 1)][:200]

bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 1 or len(x[1]) > 1]

In [9]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents([comment.split() for comment in data.ngrams_reviews])
# Filter only those that occur at least 2 times
finder.apply_freq_filter(100)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)

trigram_pmi = pd.DataFrame(trigram_scores, columns =['trigram', 'pmi'])
#trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

In [10]:
def trigram_filter(trigram):
    tag = nltk.pos_tag(trigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['JJ','NN']:
        return False
    if trigram[0] in nouns_list or trigram[-1] in nouns_list or trigram[1] in nouns_list:
        return False
    if 'n' in trigram or 't' in trigram:
         return False
    if 'PRON' in trigram:
        return False
    return True 

In [11]:
filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 trigram_filter(trigram['trigram'])\
                                                 and trigram.pmi > 1, axis = 1)][:200]


trigrams = [' '.join(x) for x in filtered_trigram.trigram.values if len(x[0]) > 1 or len(x[1]) > 1 and len(x[2]) > 1]

In [12]:
bigrams[:500]

['tl dr',
 'entertainment expo',
 'electronic entertainment',
 'alt tab',
 'sci fi',
 'false advertising',
 'user interface',
 'multi tool',
 'early access',
 'procedural generation',
 'sean murray',
 'pre order',
 'star citizen',
 'current state',
 'frame rate',
 'hype train',
 'inventory management',
 'downloadable content',
 'hello games',
 'price tag',
 'personal computer',
 'load screen',
 'solar system',
 'player character',
 'graphic card',
 'non player',
 'foundation update',
 'base building',
 'small team',
 'alien race',
 'year old',
 'look forward',
 'negative review',
 'max setting',
 'single player',
 'open world',
 'space station',
 'technical issue',
 'walk around',
 'full price',
 'alien language',
 'frame drop',
 'anyone who',
 'figure out',
 'space battle',
 'complain about',
 'performance issue',
 'star system',
 'fps drop',
 'talk about',
 'no man',
 'an indie',
 'fly around',
 'wait until',
 'future update',
 'money back',
 'interact with',
 'my opinion',
 'quintil

In [13]:
trigrams[:500]

['electronic entertainment expo',
 'role playing video',
 'massively multiplayer online',
 'non player character',
 'paid downloadable content',
 'everyone versus everyone',
 'an early access',
 'frame rate drop',
 'personal computer port',
 'player versus player',
 'field of view',
 'call of duty',
 'personal computer version',
 'frame rate issue',
 'playing video game',
 'the store page',
 'figure out how',
 'the user interface',
 'early access game',
 'it current state',
 'my personal computer',
 'the hype train',
 'the procedural generation',
 'my money back',
 'multiplayer online game',
 'no man sky',
 'same thing over',
 'on personal computer',
 'wait for sale',
 'look forward to',
 'waste of money',
 'so much potential',
 'hello games will',
 'the price tag',
 'be current state',
 'plant and animal',
 'the exact same',
 'the foundation update',
 'first few hour',
 'the frame rate',
 'hello games have',
 'man be sky',
 'after few hour',
 'point of interest',
 'sean murray be',
 '

In [14]:
# Concatenate n-grams
def replace_ngram(x):
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    return x

reviews_w_ngrams = data.copy()
reviews_w_ngrams.Preprocess = reviews_w_ngrams.ngrams_reviews.map(lambda x: replace_ngram(x))

C:\Users\researcher\AppData\Local\Temp\ipykernel_15972\1009032966.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reviews_w_ngrams.Preprocess = reviews_w_ngrams.ngrams_reviews.map(lambda x: replace_ngram(x))


In [15]:
reviews_w_ngrams = reviews_w_ngrams.Preprocess.map(lambda x: [word for word in x.split()\
                                                 if                                                      
                                                              word not in stop_word_list
                                                              and 
                                                              word not in nouns_list\
                                                              and word not in english_names\
                                                              and len(word) > 1])

In [16]:
data['ngrams_token'] = reviews_w_ngrams
data.head()

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"[review, prior, update, nothin, fail, miserabl..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"[first, player, one, point, mear]"
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"[prove, this_game, big, scam, know]"
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"[the_developer, completely, lie_about, countle..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"[clearly, strong, feel, play, hour, hello_game..."


# Remove Stopwords for LDA and NMF etc.

In [17]:
def to_lower(sent) :
    return sent.lower()

In [18]:
stopwords= set(["game","not","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "nor", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

In [19]:
lda_reviews = []
for content in tqdm(data['correctmapping']):
    content = re.sub(" \d+", " ", content).strip()
    content = re.sub('[^A-Za-z]+', ' ', content)
    content = re.sub('\s+', ' ', content)
    content = ' '.join(e.lower() for e in content.split() if e.lower() not in stopwords)
    lda_reviews.append(content.strip())

100%|███████████████████████████████████████████████████████████████████████| 163847/163847 [00:03<00:00, 53463.67it/s]


In [20]:
print (data['correctmapping'][26])
print (lda_reviews[26])

['this', 'be', 'probably', 'the', 'part', 'where', 'start', 'to', 'get', 'thumb', 'down', 'because', 'of', 'my', 'review', 'after', 'revisit', 'no', 'mans', 'sky', 'and', 'check', 'out', 'the', 'new', 'update', 'think', 'of', 'the', 'recent', 'review', 'on', 'this', 'game', 'should', 'be', 'remove']
probably part start get thumb review revisit no mans sky check new update think recent review remove


In [21]:
data['lda_reviews'] = lda_reviews
data.head()

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token,lda_reviews
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"[review, prior, update, nothin, fail, miserabl...",review prior update nothin fail miserably anno...
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"[first, player, one, point, mear]",first come player one point mear
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"[prove, this_game, big, scam, know]",prove big scam know
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"[the_developer, completely, lie_about, countle...",developer completely lie countless feature meant
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"[clearly, strong, feel, play, hour, hello_game...",clearly strong feel play hour hello games rele...


In [22]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([comment.split() for comment in data.lda_reviews])
# Filter only those that occur at least 2 times
finder.apply_freq_filter(200)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

bigram_pmi = pd.DataFrame(bigram_scores,columns= ['bigram', 'pmi'])
#bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

In [23]:
def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in nouns_list or bigram[1] in nouns_list:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True

In [24]:
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 1, axis = 1)][:200]

bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 1 or len(x[1]) > 1]

In [25]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents([comment.split() for comment in data.lda_reviews])
# Filter only those that occur at least 2 times
finder.apply_freq_filter(100)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)

trigram_pmi = pd.DataFrame(trigram_scores, columns =['trigram', 'pmi'])
#trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

In [26]:
def trigram_filter(trigram):
    tag = nltk.pos_tag(trigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['JJ','NN']:
        return False
    if trigram[0] in nouns_list or trigram[-1] in nouns_list or trigram[1] in nouns_list:
        return False
    if 'n' in trigram or 't' in trigram:
         return False
    if 'PRON' in trigram:
        return False
    return True 

In [27]:
filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 trigram_filter(trigram['trigram'])\
                                                 and trigram.pmi > 1, axis = 1)][:200]


trigrams = [' '.join(x) for x in filtered_trigram.trigram.values if len(x[0]) > 1 or len(x[1]) > 1 and len(x[2]) > 1]

In [28]:
bigrams[:500]

['tl dr',
 'entertainment expo',
 'electronic entertainment',
 'alt tab',
 'sci fi',
 'false advertising',
 'user interface',
 'early access',
 'procedural generation',
 'multi tool',
 'sean murray',
 'pre order',
 'star citizen',
 'current state',
 'gb ram',
 'frame rate',
 'hype train',
 'inventory management',
 'downloadable content',
 'hello games',
 'price tag',
 'personal computer',
 'load screen',
 'solar system',
 'player character',
 'graphic card',
 'non player',
 'foundation update',
 'base building',
 'center galaxy',
 'small team',
 'alien race',
 'waste money',
 'year old',
 'man sky',
 'learn language',
 'plant animal',
 'look forward',
 'max setting',
 'negative review',
 'open world',
 'single player',
 'space station',
 'technical issue',
 'alien language',
 'walk around',
 'full price',
 'recommend anyone',
 'center universe',
 'frame drop',
 'space battle',
 'performance issue',
 'star system',
 'fps drop',
 'write review',
 'no man',
 'fly around',
 'future update'

In [29]:
trigrams[:500]

['electronic entertainment expo',
 'role playing video',
 'massively multiplayer online',
 'non player character',
 'paid downloadable content',
 'everyone versus everyone',
 'frame rate drop',
 'personal computer port',
 'piece get piece',
 'player versus player',
 'learn alien language',
 'ship multi tool',
 'personal computer version',
 'frame rate issue',
 'no man sky',
 'get piece get',
 'get money back',
 'no man lie',
 'make feel like',
 'play no man']

In [30]:
# Concatenate n-grams
def replace_ngram(x):
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    return x

reviews_w_ngrams_lda = data.copy()
reviews_w_ngrams_lda.Preprocess = reviews_w_ngrams_lda.lda_reviews.map(lambda x: replace_ngram(x))

C:\Users\researcher\AppData\Local\Temp\ipykernel_15972\3664065759.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reviews_w_ngrams_lda.Preprocess = reviews_w_ngrams_lda.lda_reviews.map(lambda x: replace_ngram(x))


In [31]:
reviews_w_ngrams_lda = reviews_w_ngrams_lda.Preprocess.map(lambda x: [word for word in x.split()\
                                                 if                                                      
                                                              word not in stop_word_list
                                                              and 
                                                              word not in nouns_list\
                                                              and word not in english_names\
                                                              and len(word) > 1])

In [32]:
data['lda_token'] = reviews_w_ngrams_lda
data.head()

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token,lda_reviews,lda_token
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"[review, prior, update, nothin, fail, miserabl...",review prior update nothin fail miserably anno...,"[review, prior, update, nothin, fail, miserabl..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"[first, player, one, point, mear]",first come player one point mear,"[first, player, one, point, mear]"
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"[prove, this_game, big, scam, know]",prove big scam know,"[prove, big, scam, know]"
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"[the_developer, completely, lie_about, countle...",developer completely lie countless feature meant,"[developer, completely, lie, countless, featur..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"[clearly, strong, feel, play, hour, hello_game...",clearly strong feel play hour hello games rele...,"[clearly, strong, feel, play_hour, hello_games..."


In [33]:
data.to_csv('./processing_datasets/preprocessed_word_correct_token.csv',index=False)